In [87]:
import pandas as pd
import numpy as np
import plotly.express as px
import pickle

from datetime import datetime, timedelta

In [6]:
with open('data/WETH_USDC_pool_swap-2024-02-02_2024-01-31.pkl', 'rb') as fp:
    data = pickle.load(fp)

In [9]:
len(data['recipient'].unique())

1040

In [47]:
data['timestamp'] = data['timestamp'].astype(float)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 500000 entries, 2024-02-02 03:38:23 to 2024-01-31 23:35:35
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   amount0    500000 non-null  object 
 1   amount1    500000 non-null  object 
 2   amountUSD  500000 non-null  object 
 3   origin     500000 non-null  object 
 4   sender     500000 non-null  object 
 5   recipient  500000 non-null  object 
 6   timestamp  500000 non-null  float64
 7   symbol0    500000 non-null  object 
 8   symbol1    500000 non-null  object 
 9   price      500000 non-null  float64
dtypes: float64(2), object(8)
memory usage: 42.0+ MB


In [4]:
# data.to_csv('data/WETH_USDC_pool_swap-2024-02-02_2024-01-31.csv')

In [ ]:
data.head()

,amount0,amount1,amountUSD,origin,sender,recipient,timestamp,symbol0,symbol1,price
time,,,,,,,,,,
2024-02-02 03:38:23,-2301.902031,1,2303.24196925395254739746488064512,0xfa1fe308a3d99c2b6a6bb73eddcf7c550dec7b39,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,1706845103,USDC,WETH,2303.10974
2024-02-02 03:38:11,-46.038173,0.02,46.06490557507905094794929761290239,0x1229966c7ea6dbaf6cb2d1227d9206da6046dc46,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0xce16f69375520ab01377ce7b88f5ba8c48f8d666,1706845091,USDC,WETH,2303.10974
2024-02-02 03:37:11,-1543.971264,0.670733917852860781,1544.86625791779820910586062667538,0xcbd505c41eebd424156fc466cfe1e56e038d0d20,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0xcbd505c41eebd424156fc466cfe1e56e038d0d20,1706845031,USDC,WETH,2303.10974
2024-02-02 03:36:23,3380,-1.466879146542257883,3380.271570810962260033587017919912,0x1571929fe6d51bd7d6538939d81fa389a3c19d55,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,1706844983,USDC,WETH,2303.10974
2024-02-02 03:36:23,-259.266374,0.112631508742000762,259.4174556310667021049574955648599,0xf9e032cb55d0dc4e3754dce32beee23b9981773c,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xf9e032cb55d0dc4e3754dce32beee23b9981773c,1706844983,USDC,WETH,2303.10974


In [77]:
class Transaction:
    def __init__(self, record: pd.Series):
        self.amount0 = record.amount0
        self.amount1 = record.amount1
        self.timestamp = record.timestamp
        self.price = record.price

class MarketInfo:
    def __init__(self, market_prices: pd.Series, amm_delay: timedelta):
        self.market_prices = market_prices
        self.amm_delay = amm_delay

    def _search(self, query_time):
        idx = self.market_prices.index.searchsorted(query_time)
        # Get the closest entry prior to the query datetime
        return self.market_prices.iloc[max(idx-1, 0)]

    def get_price_for_trader(self, query_time):
        return self._search(query_time)

    def get_delayed_price_for_amm(self, query_time):
        delayed_time = query_time - self.amm_delay
        return self._search(delayed_time)

class ITransactFeesModel:
    """Transaction fees calculation interface"""
    def __init__(self, market_info: MarketInfo):
        self.market_info = market_info

    def calculate(self, trader, new_order: Transaction):
        pass

class IPnLModel:
    """Trader's PnL calculation interface"""
    def __init__(self, market_info: MarketInfo):
        self.market_info = market_info

    def calculate(self, new_order: Transaction, fees: int):
        pass

class SimplePnLModel(IPnLModel):
    def calculate(self, new_order: Transaction, fees: int):
        query_time = pd.to_datetime(new_order.timestamp, unit='s')
        mkt_price = self.market_info.get_price_for_trader(query_time)
        # print("JOey ", new_order.amount0, (mkt_price, new_order.price), fees)
        # return new_order.amount0 * (mkt_price - new_order.price) - fees
        return 1

class Trader:
    def __init__(self, id: str):
        self.id = id
        self.cum_pnl: list[float] = [0.0]
        self.history: list[Transaction] = []
    
    def on_new_trade(self, trade: pd.Series, fees_model: ITransactFeesModel, pnl_model: IPnLModel):
        self.history.append(Transaction(trade))
        fees = fees_model.calculate(self, trade)
        tmp_pnl = self.cum_pnl[-1] + pnl_model.calculate(trade, fees)
        self.cum_pnl.append(tmp_pnl)

    def get_current_pnl(self):
        return self.cum_pnl[-1]

class AMMSimulator:
    """Simulation of automatic market maker transactions"""

    def __init__(self, market_info: MarketInfo, fees_model: ITransactFeesModel,
                 pnl_model: IPnLModel, transactions_data: pd.DataFrame):
        """Initialize a new instance of the AMMSimulator class."""
        self.market_info = market_info
        self.fees_model = fees_model
        self.pnl_model = pnl_model
        self.raw_tx_data = transactions_data
        self.traders = dict()

        self.raw_tx_data.sort_values('timestamp')

    def _datetime_filter(self, start, end):
        return (self.raw_tx_data.index >= start) & (self.raw_tx_data.index <= end)
    
    def _run_trader_records(self, trader_records: pd.DataFrame):
        trader_id = trader_records['sender'].iloc[0]
        print(f"Processing trader {trader_id}")
        trader = self.traders.setdefault(trader_id, Trader(trader_id))
        for idx, record in trader_records.iterrows():
            trade = Transaction(record)
            trader.on_new_trade(trade, self.fees_model, self.pnl_model)
        
    def replay(self, start_time=None, end_time=None):
        """Replay past transactions and summarize participants' PnL"""
        start_time = start_time or self.raw_tx_data.index[0]
        end_time = end_time or self.raw_tx_data.index[-1]

        records = self.raw_tx_data
        # print(self.raw_tx_data.info())
        records.groupby('sender').apply(self._run_trader_records)

        print(sorted([t.get_current_pnl() for t in self.traders.values()])[-10:])


class SimpleTransactFeesModel(ITransactFeesModel):
    def calculate(self, trader, new_order: Transaction):
        fees_factor = max(0, trader.get_current_pnl()*0.002)
        # print(f'model: {new_order.amount0}, {fees_factor}, {new_order.amount0 * fees_factor}!!')
        # return new_order.amount0 * fees_factor
        return 0
    

# Create random extermal market prices
mu, sigma = data['price'].mean(), data['price'].std()
mock_mkt_data = np.random.normal(loc=mu, scale=sigma, size=len(data))
mock_mkt_data = pd.Series(mock_mkt_data, index=data.index)

amm_delay = timedelta(seconds=3)
market_info = MarketInfo(mock_mkt_data, amm_delay)

# Initialize the fees and pnl models
tx_fees_model = SimpleTransactFeesModel(market_info)
pnl_model = SimplePnLModel(market_info)
    
# Run simulation
amm_sim = AMMSimulator(market_info, tx_fees_model, pnl_model, data)
amm_sim.replay()


Processing trader 0x000000000000f16dda43d8d2add50769205e8619
Processing trader 0x00000000000ba9cd9f5175108141a82b6c24d727
Processing trader 0x0000000000450702bc4f750fd1e7ecad7054c4f1
Processing trader 0x00000000009e50a7ddb7a7b0e2ee6604fd120e49
Processing trader 0x0000000000a84d1a9b0063a910315c7ffa9cd248
Processing trader 0x0096913d68a000311266250f0000000000e04c00
Processing trader 0x01bd2da640345f1c29831b7cef9a434298408172
Processing trader 0x03f911aedc25c770e701b8f563e8102cfacd62c0
Processing trader 0x073197fa2656bdaf1ca018b7b333379683b1d8ad
Processing trader 0x0a7ec264d69f8823b2c8467431b8bf46ba6f853c
Processing trader 0x0b8a49d816cc709b6eadb09498030ae3416b66dc
Processing trader 0x0ddc6f9ce13b985dfd730b8048014b342d1b54f7
Processing trader 0x0e464efa2a2d1ddea113ded88a6a2b7b380c1cfe
Processing trader 0x1111111254eeb25477b68fb85ed929f73a960582
Processing trader 0x1111111254fb6c44bac0bed2854e76f90643097d
Processing trader 0x14f2b6ca0324cd2b013ad02a7d85541d215e2906
Processing trader 0x1661

In [ ]:
top_10_pnl = sorted([pd.Series(t.cum_pnl, name=t.id) for t in amm_sim.traders.values()], key=lambda s: s.iloc[-1])[-10:]
fig = px.line(top_10_pnl)
fig.show()